In [33]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## **CONFIG**

In [34]:
import os
import yaml

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt

from data_preparation import transform_data

In [36]:
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

IMG_DIR = config['data']['data_dir']
ANNOS_FILE = config['data']['annos_file']
BATCH_SIZE = config['training']['batch_size']
IMG_SIZE = (config['data']['image_size'], config['data']['image_size'])

TRANSFORM = transform_data(IMG_SIZE)
DEVICE = torch.device('cuda') if torch.cuda.is_available else 'cpu'

LR = config['training']['learning_rate']
GAMMA = config['scheduler']['gamma']
STEP_SIZE = config['scheduler']['step_size']

DISCRIMINANT_CRITERION = nn.CrossEntropyLoss()

NUM_EPOCHS = config['training']['num_epochs']

## **PRE-PROP**

In [ ]:
from data_preparation import generate_json_annotations

generate_json_annotations(IMG_DIR, ANNOS_FILE)

## **DATASET \& LOADERS**

In [38]:
from data_preparation import ImageDataset, train_val_dataset, train_val_dataloader

In [39]:
DATASET = ImageDataset(annotations_file='annotations_classif.json',
                      img_dir=IMG_DIR,
                      transform=TRANSFORM)

TRAIN_TEST_SET = train_val_dataset(DATASET, 0.1)
TRAIN_SET = TRAIN_TEST_SET['train']
TRAIN_VAL_SET = train_val_dataset(TRAIN_SET, 0.2)
TRAIN_VAL_LOADER = train_val_dataloader(TRAIN_VAL_SET, BATCH_SIZE)

TEST_SET = TRAIN_TEST_SET['val']

# **TRAINING**

In [40]:
from train import evalutrain_model
from models import FineTunedResNet18, GBZ

## **GBZ**

In [41]:
# Hyperparameters
input_channels = 1
output_channels = 1
latent_dim = 100

model = GBZ(latent_dim, num_classes=6).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [ ]:
fit_model, train_losses, train_accs, val_losses, val_accs = evalutrain_model(model,
                'vae_gbz',
                TRAIN_VAL_LOADER,
                TRAIN_VAL_SET,
                DISCRIMINANT_CRITERION,
                optimizer,
                scheduler,
                DEVICE,
                num_epochs=NUM_EPOCHS)

### Inférence

In [ ]:
from evaluation import plot_acc_train
plot_acc_train(train_losses,
               train_accs,
               val_losses,
               val_accs,
               type_model='vae_gbz')

In [ ]:
from evaluation import test_evaluation

true_labels, pred_labels, probs_positive, true_logs = test_evaluation(fit_model, 
                                                            TEST_SET,
                                                            device='cpu',
                                                            type_model='vae_gbz')

In [ ]:
from evaluation import plot_confusion_matrix

cm = plot_confusion_matrix(true_labels, pred_labels, attack_name='vae_gbz')

## **DISCRIMINANT**

In [13]:
# Hyperparameters
input_channels = 1
output_channels = 1
latent_dim = 20

# model = GBZ(input_channels, latent_dim, num_classes=6, output_channels=output_channels, image_size=68).to(DEVICE)
model = FineTunedResNet18().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [ ]:
fit_model, train_losses, train_accs, val_losses, val_accs = evalutrain_model(model,
                'convnet',
                TRAIN_VAL_LOADER,
                TRAIN_VAL_SET,
                DISCRIMINANT_CRITERION,
                optimizer,
                scheduler,
                DEVICE,
                num_epochs=NUM_EPOCHS)

### Inference

In [ ]:
from evaluation import plot_acc_train
plot_acc_train(train_losses,
               train_accs,
               val_losses,
               val_accs,
               type_model='convnet')

In [ ]:
from evaluation import test_evaluation

true_labels, pred_labels, probs_positive, true_logs = test_evaluation(fit_model, 
                                                            TEST_SET,
                                                            device='cpu',
                                                            type_model='convnet')

In [ ]:
from evaluation import plot_confusion_matrix

cm = plot_confusion_matrix(true_labels, pred_labels)

## **PERFORM ATTACKS**

***

When executing the below cells, please update `model_type` parameter to which model you are using, and which `attack` you want to perform.

### **CONTRAST**

In [ ]:
from evaluation import perform_attack

adv_images_contrast = perform_attack(fit_model, TEST_SET, model_type='convnet', attack='contrast', epsilon=0.1)

In [ ]:
true_labels, pred_labels_attack_contrast, probs_positive, logits_convnet_contrast = test_evaluation(fit_model, 
                                                            TEST_SET,
                                                            device='cpu',
                                                            type_model='convnet',
                                                            attack=True,
                                                            adv_images=adv_images_contrast
                                                            )

In [ ]:
from PIL import Image
import numpy as np

true_images = np.array([img for img, _ in TEST_SET])
adv_images_bg = np.array([img.cpu().detach().numpy() for img in adv_images_contrast])
i = 5678

# plot images
plt.figure(figsize=(10, 10))
plt.subplot(1, 3, 1)
plt.imshow(adv_images_bg[i].squeeze(), cmap='gray')
plt.title('Adversarial Image')

plt.subplot(1, 3, 2)
plt.imshow(true_images[i].squeeze(), cmap='gray')
plt.title('True Image')

plt.subplot(1, 3, 3)
plt.imshow(true_images[i].squeeze() - adv_images_bg[i].squeeze(), cmap='gray')
plt.title('Difference')
plt.show()

In [ ]:
cm = plot_confusion_matrix(true_labels, pred_labels_attack_contrast)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(true_labels, pred_labels_attack_contrast)

### **BRIGHTNESS**

In [ ]:
from evaluation import perform_attack

epsilons = [0.01, 0.03, 0.05, 0.08, 0.1]
accuracy_scores_convnet = []
adv_images_bright = []
for eps in epsilons:
    adv_images = perform_attack(fit_model, TEST_SET, model_type='convnet', attack='brightness', epsilon=eps)
    true_labels, pred_labels_attack, probs_positive, logs = test_evaluation(fit_model, 
                                                            TEST_SET,
                                                            device='cpu',
                                                            type_model='convnet',
                                                            attack=True,
                                                            adv_images=adv_images
                                                            )
    accuracy_scores_convnet.append(accuracy_score(true_labels, pred_labels_attack))
    adv_images_bright.append(adv_images)
    print(f'accuracy score with attack: {accuracy_score(true_labels, pred_labels_attack)} \n')

In [ ]:
true_labels, pred_labels_attack_bright, probs_positive, logits_convnet_br = test_evaluation(fit_model, 
                                                            TEST_SET,
                                                            device='cpu',
                                                            type_model='convnet',
                                                            attack=True,
                                                            adv_images=adv_images_bright
                                                            )

In [ ]:
from PIL import Image
import numpy as np

true_images = np.array([img for img, _ in TEST_SET])
adv_images_bg = np.array([img.cpu().detach().numpy() for img in adv_images_bright])
i = 4000

# plot images
plt.figure(figsize=(10, 10))
plt.subplot(1, 3, 1)
plt.imshow(adv_images_bg[i].squeeze(), cmap='gray')
plt.title('Adversarial Image')

plt.subplot(1, 3, 2)
plt.imshow(true_images[i].squeeze(), cmap='gray')
plt.title('True Image')

plt.subplot(1, 3, 3)
plt.imshow(true_images[i].squeeze() - adv_images_bg[i].squeeze(), cmap='gray')
plt.title('la grosse Difference')
plt.show()

In [ ]:
cm = plot_confusion_matrix(true_labels, pred_labels_attack_bright)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(true_labels, pred_labels_attack_bright)

### **ONE PIXEL DISCRIMINANT**

This part is to perform ONE PIXEL ATTACK on the discriminant ResNet18 Model.

In [ ]:
from attacks import run_one_pixel_attack
import numpy as np
import os

list_labels = []
for name in os.listdir(IMG_DIR):
    list_labels.append(name)
list_labels = np.sort(list_labels)
test_loader = torch.utils.data.DataLoader(TEST_SET, batch_size=1, shuffle=False)
fit_model = fit_model.to(DEVICE)
adv_images_pixel = run_one_pixel_attack(fit_model, test_loader, list_labels, model_type='convnet')

In [ ]:
# adv_images_pixel = torch.tensor(adv_images_pixel)
true_labels, pred_labels_attack_pixel, probs_positive, logits_convnet_pixel = test_evaluation(fit_model, 
                                                            TEST_SET,
                                                            device='cpu',
                                                            type_model='convnet',
                                                            attack=True,
                                                            adv_images=adv_images_pixel
                                                            )

In [ ]:
cm = plot_confusion_matrix(true_labels, pred_labels_attack_pixel)

### **FSGM ATTACK**

In [ ]:
# fsgm attack
from evaluation import perform_attack
from sklearn.metrics import accuracy_score

epsilons = [0.00,0.01, 0.03, 0.05, 0.08, 0.1]
accuracy_scores_convnet = []
tot_adv_images = []
for eps in epsilons:
    adv_images = perform_attack(fit_model, TEST_SET, model_type='vae_gbz', attack='pgd', epsilon=eps)
    true_labels, pred_labels_attack, probs_positive, logs = test_evaluation(fit_model, 
                                                            TEST_SET,
                                                            device='cpu',
                                                            type_model='vae_gbz',
                                                            attack=True,
                                                            adv_images=adv_images
                                                            )
    accuracy_scores_convnet.append(accuracy_score(true_labels, pred_labels_attack))
    tot_adv_images.append(adv_images)
    print(f'accuracy score with attack: {accuracy_score(true_labels, pred_labels_attack)} \n')

    

In [ ]:
# plot images, where the big figure x axis depends on epsilon and y axis the different images
fig, axs = plt.subplots(6, 6, figsize=(10, 10))
for i in range(6):
    for j in range(6):
        axs[i, j].imshow(tot_adv_images[j][i].detach().numpy().squeeze(), cmap='gray')
        if i == 0:
            axs[i, j].set_title(f'epsilon: {epsilons[j]}', fontsize=8)
        axs[i, j].axis('off')
       
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig('./Inference/pgd_attacks_test_images_vae_gbz.pdf')
plt.show()


In [ ]:
cm = plot_confusion_matrix(true_labels, pred_labels_attack)

In [ ]:
# compute accuracy
from sklearn.metrics import accuracy_score

print(f'accuracy score with attack: {accuracy_score(true_labels, pred_labels_attack)} \n')
print(f'accuracy score without attack: {accuracy_score(true_labels, pred_labels)} \n')


In [186]:
import numpy as np

true_images = np.array([img for img, _ in TEST_SET])
adv_images = np.array([img.cpu().detach().numpy() for img in adv_images])

In [ ]:
from PIL import Image

i = 4000

# plot images
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(adv_images[i].squeeze(), cmap='gray')
plt.title('Adversarial Image')

plt.subplot(1, 2, 2)
plt.imshow(true_images[i].squeeze(), cmap='gray')
plt.title('True Image')
plt.show()

### **ONE PIXEL ATTACK**

This part is to perform ONE PIXEL ATTACK.

In [ ]:
from attacks import run_one_pixel_attack


list_labels = []
for name in os.listdir(IMG_DIR):
    list_labels.append(name)
list_labels = np.sort(list_labels)
test_loader = torch.utils.data.DataLoader(TEST_SET, batch_size=1, shuffle=False)
fit_model = fit_model.to(DEVICE)
adv_images_pixel = run_one_pixel_attack(fit_model, test_loader, list_labels)

In [226]:
adv_images_pixel = np.array([img.cpu().detach().numpy() for img in adv_images_pixel])

In [ ]:
adv_img_pixel = np.array(adv_images_pixel).squeeze(1)
i = 5

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(adv_img_pixel[i].squeeze(), cmap='gray')
plt.title('Adversarial Image')

plt.subplot(1, 2, 2)
plt.imshow(true_images[i].squeeze(), cmap='gray')
plt.title('True Image')
plt.show()

In [ ]:
adv_images_pixel = torch.tensor(adv_images_pixel).to('cpu')
true_labels, pred_labels_attack_pixel, probs_positive, logits = test_evaluation(fit_model, 
                                                            TEST_SET,
                                                            device='cpu',
                                                            type_model='vae_gbz',
                                                            attack=True,
                                                            adv_images=adv_images_pixel
                                                            )

In [ ]:
logits_pixel = torch.max(torch.Tensor(logits).squeeze(), dim=1)
true_logits = torch.max(torch.Tensor(true_logs).squeeze(), dim=1)

print(f'error rate with pixel attack: {np.array(logits_pixel) / np.array(true_logits) / 100} \n')

### **PGD ATTACK**

This part is to perform PGD ATTACK.

In [ ]:
adv_images = perform_attack(fit_model, TEST_SET, model_type='vae_gbz', attack='pgd')

In [ ]:
true_labels, pred_labels_pgd, probs_positive, logs = test_evaluation(fit_model, 
                                                            TEST_SET,
                                                            device='cpu',
                                                            type_model='vae_gbz',
                                                            attack=True,
                                                            adv_images=adv_images
                                                            )

In [ ]:
print(f'accuracy score with attack: {accuracy_score(true_labels, pred_labels_pgd)} \n')

### **CW ATTACK**

In [ ]:
adv_images = perform_attack(fit_model, TEST_SET, model_type='convet', attack='cw')

In [ ]:
true_labels, pred_labels_cw, probs_positive, logs = test_evaluation(fit_model, 
                                                            TEST_SET,
                                                            device='cpu',
                                                            type_model='convnet',
                                                            attack=True,
                                                            adv_images=adv_images
                                                            )

In [ ]:
print(f'accuracy score with attack: {accuracy_score(true_labels, pred_labels_cw)} \n')

## **PLOT LATENT SPACE WITH PCA**

In [35]:
for x, y in TEST_SET:
    z = fit_model.get_z(x.unsqueeze(0).to(DEVICE))
    break

In [ ]:
from utils import plot_latent_space

plot_latent_space(fit_model, TEST_SET, device='cuda')

In [ ]:
list1 = [0.998, 0.98, 0.94, 0.88, 0.81, 0.78]
list2 = [0.999, 0.84, 0.48, 0.41, 0.36, 0.33]
epsilons = [0, 0.01, 0.03, 0.05, 0.08, 0.1]

fig = plt.figure(figsize=(6, 4))
plt.plot(epsilons, list1, "-", label='gbz', color='mediumslateblue')
plt.plot(epsilons, list2, "-", label='resnet18', color='mediumvioletred')
plt.scatter(epsilons, list1, marker='*', color='mediumslateblue', s=100)
plt.scatter(epsilons, list2, marker='*', color='mediumvioletred', s=100)
plt.yticks(np.arange(0.3, 1.1, step=0.2))
plt.grid(True)
plt.xlabel(r"$\epsilon \in \{0.01, 0.03, 0.05, 0.08, 0.1\}$")
plt.ylabel("Accuracy")
plt.legend()
plt.tight_layout()
plt.savefig('./Inference/accs_models_fsgm.pdf')
